In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [ ]:
train_folder = "~/neurotech/grasp-and-lift/train"

In [ ]:
data_list = []

for subject in range(1, 13):
    for series in range(1, 9):
        data_file = os.path.join(train_folder, f"subj{subject}_series{series}_data.csv")
        events_file = os.path.join(train_folder, f"subj{subject}_series{series}_events.csv")
        data_df = pd.read_csv(data_file)
        events_df = pd.read_csv(events_file)
        merged_df = pd.merge(data_df, events_df, on="id")
        data_list.append(merged_df)
        print(f'Loaded data for subject {subject} series {series}')

combined_df = pd.concat(data_list, ignore_index=True)
print(f"\nCombined DataFrame shape: {combined_df.shape}")

In [ ]:
id_parts = combined_df["id"].str.split("_", expand=True)

combined_df["subject"] = id_parts[0]
combined_df["series"] = id_parts[1]
combined_df["time_point"] = id_parts[2]
combined_df["time_point"] = combined_df["time_point"].astype(int)
combined_df['subject_series'] = combined_df['subject'] + "_" + combined_df['series']

In [ ]:
time_threshold = 100000 
train_data = []
test_data = []

for subject_series in combined_df['subject_series'].unique():
    subject_series_data = combined_df[combined_df['subject_series'] == subject_series]
    subject_series_data = subject_series_data.sort_values('time_point')
    train_subset = subject_series_data[subject_series_data['time_point'] <= time_threshold]
    test_subset = subject_series_data[subject_series_data['time_point'] > time_threshold]
    train_data.append(train_subset)
    test_data.append(test_subset)
train_df = pd.concat(train_data, ignore_index=True)
test_df = pd.concat(test_data, ignore_index=True)

In [ ]:
for col in train_df.columns:
    print(col)

In [ ]:
electrode_columns = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FC5", "FC1", "FC2", "FC6", "T7", "C3", "Cz", "C4", "T8", "TP9", "CP5", "CP1", "CP2", "CP6", "TP10", "P7", "P3", "Pz", "P4", "P8", "PO9", "O1", "Oz", "O2", "PO10"]
event_columns = ["HandStart", "FirstDigitTouch", "BothStartLoadPhase", "LiftOff", "Replace", "BothReleased"]
lookback = 10

In [ ]:
X_train_seq = []
y_train_seq = []

In [ ]:
for i in range(lookback, len(train_df)):
    X_train_seq.append(train_df.iloc[i-lookback:i][electrode_columns].values)
    y_train_seq.append(train_df.iloc[i][event_columns].values)

In [ ]:
X_train_seq = np.array(X_train_seq)
y_train_seq = np.array(y_train_seq)

X_test_seq = []
y_test_seq = []

In [ ]:
for i in range(lookback, len(test_df)):
    X_test_seq.append(test_df.iloc[i-lookback:i][electrode_columns].values)
    y_test_seq.append(test_df.iloc[i][event_columns].values)

In [ ]:
X_test_seq = np.array(X_test_seq)
y_test_seq = np.array(y_test_seq)

scaler = StandardScaler()
X_train_seq_scaled = scaler.fit_transform(X_train_seq.reshape(-1, X_train_seq.shape[-1])).reshape(X_train_seq.shape)
X_test_seq_scaled = scaler.transform(X_test_seq.reshape(-1, X_test_seq.shape[-1])).reshape(X_test_seq.shape)

In [ ]:
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell
import tensorflow as tf
from ncps import wirings
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping, ModelCheckpoint
import kerasncp as kncp
from keras.utils.np_utils import to_categorical

In [ ]:
num_classes = len(event_columns)
timesteps = lookback
input_shape = (timesteps, len(electrode_columns))

wiring = wirings.AutoNCP(100, 25)
rnn_cell = LTCCell(wiring)

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=input_shape),
    keras.layers.Conv1D(filters=82, kernel_size=3, activation='relu', padding='causal'),
    keras.layers.RNN(rnn_cell, return_sequences=True),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(num_classes, activation="sigmoid")
])

model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=1, mode='max')
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(
    X_train_seq_scaled,
    y_train_seq,
    epochs=1000,
    batch_size=30,
    validation_data=(X_test_seq_scaled, y_test_seq),
    shuffle=True,
    callbacks=[early_stopping, model_checkpoint]
)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_seq_scaled, y_test_seq)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
y_pred = model.predict(X_test_seq_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)
report = classification_report(y_test_seq, y_pred_binary, target_names=event_columns)
print("Classification Report:")
print(report)